In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import split
import pandas as pd
import matplotlib.pyplot as plt
import upsetplot
from upsetplot import UpSet
#créer une session dans le master
spark = SparkSession.builder \
    .master("spark://172.20.53.96:7077") \
    .appName("WDC-complete") \
    .config("spark.executor.memory","28g") \
    .config("spark.driver.memory","28g") \
    .getOrCreate()
# spark = SparkSession.builder.master("local").appName("WDC-complete").getOrCreate()

spark.conf.set("spark.worker.cleanup.enabled",True)
spark.conf.set("spark.worker.cleanup.interval",1800)
spark.conf.set("spark.worker.cleanup.appDataTtl",3600)
spark.conf.set("spark.sql.shuffle.partitions",1000)

#fichiers de config qui permettent de se connecter au serveur de stockage s3 qui contient les fichiers de DataCommons
endpoint_url = 'https://s3.os-bird.glicid.fr/'
aws_access_key_id = 'bbd95ea3c1174caa88345404b84e458f'
aws_secret_access_key = 'eaf2a72ecf9845f583af7f3513c44f25'
hadoopConf = spark._jsc.hadoopConfiguration()
hadoopConf.set('fs.s3a.access.key', aws_access_key_id)
hadoopConf.set('fs.s3a.secret.key', aws_secret_access_key)
hadoopConf.set('fs.s3a.endpoint', endpoint_url)
hadoopConf.set('fs.s3a.path.style.access', 'true')
hadoopConf.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')

hadoopConf.set('spark.worker.cleanup.enabled', 'true')
hadoopConf.set('fs.s3a.committer.name', 'magic')

In [ ]:
# read = spark.read.option("header", True).csv(f"s3a://test-out/{repertory}/hashdom{i}-{parts}")
read = spark.read.option("header", True).csv(f"s3a://test-out/types/covavevolutionresult")

read.show(truncate=110)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from pyspark.sql.functions import col

# dans le dataframe des résultats , remplace les valeurs null des colonne "percentage_count_evolution","percentage_average_evolution" et "percentage_coverage_evolution" par 0
DF_final = read.withColumn("percentage_count_evolution", col("percentage_count_evolution").cast("double")).fillna(0, subset=["percentage_count_evolution"])
DF_final = DF_final.withColumn("percentage_average_evolution", col("percentage_average_evolution").cast("double")).fillna(0, subset=["percentage_average_evolution"])
DF_final = DF_final.withColumn("percentage_coverage_evolution", col("percentage_coverage_evolution").cast("double")).fillna(0, subset=["percentage_coverage_evolution"])


print("test")
resultat = DF_final[DF_final['type'] == 'isa:<schema.org/invoice>']
resultat.show(truncate=110)

#DF_final.show(truncate=110)

#Graphique affichant les 15 types de schema.org qui ont le plus de pourcentage d'augmentation dans leur nombre d'occurence entre 2022 et 2023
orderedDF_by_count = DF_final.orderBy(col('percentage_count_evolution').desc())
df_pd_count = orderedDF_by_count.limit(15).toPandas()
ax_count=df_pd_count.plot(kind='bar' , x='type', y ='percentage_count_evolution')
ax_count.set_title('Top 15 most structured types with the best percentage evolution of count')
plt.show()

'''orderedDF_by_count = DF_final.orderBy(col('percentage_count_evolution').desc())
df_pd_count = orderedDF_by_count.limit(16).toPandas()
df_pd_count = df_pd_count.drop(0, axis=0)

#print(df_pd_count)

ax_count=df_pd_count.plot(kind='bar' , x='type', y ='percentage_count_evolution')
ax_count.set_title('Top 15 most structured types with the best percentage evolution of count')
plt.show()'''


#Graphique affichant les 15 types de schema.org qui ont le plus de pourcentage d'augmentation dans leur average entre 2022 et 2023
#orderedDF_by_average = DF_final.orderBy(col('percentage_average_evolution').desc())
orderedDF_by_average = DF_final.orderBy(col('percentage_average_evolution').desc())
print("average")
#orderedDF_by_average = orderedDF_by_average.groupBy('type')
orderedDF_by_average.show(truncate=110)



'''test = orderedDF_by_average[orderedDF_by_average['type'] == 'isa:<schema.org/Oncologic>']
print("test")
print(test)
test.show(truncate=110)'''

df_pd_average = orderedDF_by_average.limit(15).toPandas()
ax_average=df_pd_average.plot(kind='bar' , x='type', y ='percentage_average_evolution')
ax_average.set_title('Top 15 most structured types with the best percentage evolution of average')
plt.show()

#Graphique affichant les 15 types de schema.org qui ont le plus de pourcentage d'augmentation dans leur coverage entre 2022 et 2023
orderedDF_by_coverage = DF_final.orderBy(col('percentage_coverage_evolution').desc())
#print("coverage")
#orderedDF_by_coverage.show(truncate=110)

df_pd_coverage = orderedDF_by_coverage.limit(15).toPandas()
ax_coverage=df_pd_coverage.plot(kind='bar' , x='type', y ='percentage_coverage_evolution')
ax_coverage.set_title('Top 15 most structured types with the best percentage evolution of coverage')
plt.show()